In [ ]:
#Collect Data from snscrape to 
try:
    import pymongo
    from pymongo import MongoClient
    import pandas as pd
    import json
    import snscrape.modules.twitter as sntwitter
    import pandas as pd
except Exception as e:
    print("Some Modules are Missing ",e)

class tweetCollection(object):
    query = 'ChatGPT since:2022-12-01 until:2023-04-11 , lang:en'
    limit = 10000
    tweets_1 = []
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if len(tweets_1) == limit:
            break
        else:
            tweets_1.append([tweet.id,
                             tweet.user.username,
                             tweet.user.verified,
                             tweet.user.location,
                             tweet.user.followersCount,
                             tweet.rawContent,
                             tweet.date,
                             tweet.retweetedTweet,
                             tweet.lang])
    df_1 = pd.DataFrame(tweets_1, columns=['UserId',
                                           'UserName',
                                           'Verified',
                                           'Location',
                                           'Followers',
                                           'Tweet',
                                           'Date',
                                           'Retweeted',
                                           'Language'])
    df_1.to_csv('Twitter_Data.csv', index=False)

if __name__ == "__main__":
    tweetCollection()
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    mydb = myclient["mydb"]
    mycol = mydb["mycollection"]
    df=pd.read_csv("Twitter_Data.csv",low_memory=False)
    data=df.to_dict(orient="records")
    mycol.insert_many(data)
    print("Inserted Data")


In [ ]:
#To get Data from MongoDB to csv file
class MongoDB(object):

    def __init__(self, dBName=None, collectionName=None):

        self.dBName = dBName
        self.collectionName = collectionName
        self.client = MongoClient("localhost", 27017, maxPoolSize=50)
        self.DB = self.client[self.dBName]
        self.collection = self.DB[self.collectionName]

    def dbFetch(self, dBName=None, collectionName=None):
        myclient = pymongo.MongoClient("mongodb://localhost:27017/")
        mydb = myclient["mydb"]
        mycol = mydb["mycollection"]
        x = mycol.find()
        tweets_1 = []
        for tweet in x:
            tweets_1.append([tweet['UserId'],
                             tweet['UserName'],
                             tweet['Verified'],
                             tweet['Location'],
                             tweet['Followers'],
                             tweet['Tweet'],
                             tweet['Timestamp'],
                             tweet['Retweeted'],
                             tweet['Language']])
            
            
            df_1 = pd.DataFrame(tweets_1, columns=['UserId',
                                           'UserName',
                                           'Verified',
                                           'Location',
                                           'Followers',
                                           'Tweet',
                                           'Date',
                                           'Retweeted',
                                           'Language'])
            df_1.to_csv('Twitter_Data.csv', index=False)
            print(tweet)
        


if __name__ == "__main__":
    mongodb = MongoDB(dBName = 'mydb', collectionName='mycollection')
    mongodb.dbFetch()
    print("Downloaded")

In [ ]:
try:
    import pymongo
    from pymongo import MongoClient
    import pandas as pd
    import json
    import snscrape.modules.twitter as sntwitter
    import pandas as pd
except Exception as e:
    print("Some Modules are Missing ",e)
    
#Read data from csv stored from Mongodb
df_tweets = pd.read_csv('Twitter_Data.csv')
df_tweets.shape

In [ ]:
df_tweets = df_tweets.drop_duplicates()
df_tweets.shape

In [ ]:
df_tweets.info()

In [ ]:
#Divide the datetime columns into two separate columns
df_tweets['Timestamp']=pd.to_datetime(df_tweets['Date'])
df_tweets['Date'] = df_tweets['Timestamp'].dt.date
df_tweets['Time_of_the_day'] = df_tweets['Timestamp'].dt.time

In [ ]:
df_tweets[df_tweets['Timestamp'].isna()]

In [ ]:
#Remove missing values
df_tweets.dropna(subset=['Timestamp'], inplace = True)

In [ ]:
# Save data for EDA 
df_tweets.to_csv('Twitter_EDA.csv', index = False)

In [ ]:
try:
    import re
    import nltk
    from nltk.corpus import stopwords
    from nltk.stem import WordNetLemmatizer
except Exception as e:
    print("Some Modules are Missing ",e)
    

In [ ]:
df_en = df_tweets[df_tweets['Language']=='en'].reset_index(drop=True)
df_en.shape

In [ ]:
df_en = df_en.drop(columns = ['Language'])

In [ ]:
#Remove URL from tweet text
df_en['Tweet'] = df_en['Tweet'].apply(lambda x: re.sub(r'http\S+', '',x))
#Remove mention (@user)
df_en['Tweet'] = df_en['Tweet'].apply(lambda x: re.sub(r'@\w+', '',x))
#All lowercases
df_en['Tweet'] = df_en['Tweet'].apply(lambda x: ' '.join(x.lower() for x in x.split()))
#Remove Punctuation
df_en['Tweet_punc'] = df_en['Tweet'].apply(lambda x: re.sub('[^\w\s]', '', x))

In [ ]:
#Replace 'chat gpt' with 'chatgpt'
df_en['Tweet_punc'] = df_en['Tweet_punc'].apply(lambda x: re.sub(r'chat gpt', 'chatgpt', x))

In [ ]:
import nltk
import ssl
from nltk.corpus import stopwords


try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

In [ ]:
stop = stopwords.words('english')
df_en['Tweet_stop'] = df_en['Tweet_punc'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop))

In [ ]:
df_en['Tweet_tokenized'] = df_en['Tweet_stop'].apply(lambda x: re.split('\W+', x))

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
from nltk import pos_tag
from nltk.corpus import wordnet

In [ ]:
#define a function to map POS tags from treebank tag into wordnet tags
wnl = WordNetLemmatizer()
def get_wordnet(pos):
    if pos.startswith('N'): 
        return wordnet.NOUN
    elif pos.startswith('V'):
        return wordnet.VERB
    elif pos.startswith('J'):
        return wordnet.ADJ
    elif pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN     #default to noun if no match

    
# Define a function to tag and lemmatize a text string    
def lemmatizer(text):
    pos_tags = pos_tag(text)
    text = [wnl.lemmatize(word, pos = get_wordnet(pos)) for word, pos in pos_tags]
    return text

df_en['Tweet_lemmatized'] = df_en['Tweet_tokenized'].apply(lambda x: lemmatizer(x))

In [ ]:
#!pip install spacy
#!python -m spacy download en_core_web_sm
#!pip install --upgrade dataclasses
#import spacy
# try spacy lemmatizer
###
def spacy_lemmatizer(text):
    # only need tagger, no need for parser and named entity recognizer, for faster implementation
    nlp = spacy.load('en_core_web_sm')
    allowed_tags=['NOUN', 'ADJ', 'VERB', 'ADV']
    doc = nlp(' '.join(text))
    lemmas = [token.lemma_ for token in doc]
    return lemmas
#df_en['Tweet_lemmatized_spacy'] = df_en['Tweet_tokenized'].apply(lambda x: spacy_lemmatizer(x))
###

In [ ]:
df_en

In [ ]:
#Remove columns not needed.
df_en_clean = df_en.drop(columns=['Tweet', 'Tweet_punc', 'Tweet_stop', 'Tweet_tokenized'])

In [ ]:
df_en_clean['Tokens'] = df_en_clean['Tweet_lemmatized'].apply(lambda x: ' '.join(x))

In [ ]:
df_en_clean.to_csv('Twitter_eng_clean.csv', index=False)